In [648]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

In [649]:
train = pd.read_csv('../input/tp2datostrain/train.csv',parse_dates=['fecha']).drop(columns=['lat','lng'])
test = pd.read_csv('../input/tp2datostrain/test.csv',parse_dates=['fecha']).drop(columns=['lat','lng'])

In [650]:
#Arreglo problema de metros totales y metros cubiertos
train['metrostotales_reales'] = train['metrostotales'].where(train.metrostotales>=train.metroscubiertos,train.metroscubiertos)
train.metroscubiertos = train['metroscubiertos'].where(train.metrostotales>=train.metroscubiertos,train.metrostotales)
train.metrostotales = train.metrostotales_reales
train.drop(columns=['metrostotales_reales'],inplace=True)

In [651]:
#Arreglo problema de metros totales y metros cubiertos
test['metrostotales_reales'] = test['metrostotales'].where(test.metrostotales>=test.metroscubiertos,test.metroscubiertos)
test.metroscubiertos = test['metroscubiertos'].where(test.metrostotales>=test.metroscubiertos,test.metrostotales)
test.metrostotales = test.metrostotales_reales
test.drop(columns=['metrostotales_reales'],inplace=True)

In [652]:
train.tipodepropiedad.isnull().sum()

46

In [653]:
cat_features = ['tipodepropiedad', 'provincia', 'ciudad']
num_features = ['banos','garages','habitaciones','metrostotales','metroscubiertos','antiguedad','adicionales']
bool_features = ['gimnasio','usosmultiples','piscina','escuelascercanas','centroscomercialescercanos'
                ,'jardin','vigilancia','buenaUbicacion','lavadero','jacuzzi']

In [654]:
#train.descripcion.sample(10)

In [655]:
pd.set_option('display.max_colwidth',-1)
import numpy as np

train.descripcion.fillna('',inplace=True)
train['jardin'] = (train.descripcion.str.contains('jardin') | train.descripcion.str.contains('jardín')).astype(int)
train['vigilancia'] = (train.descripcion.str.contains('vigilancia') | train.descripcion.str.contains('seguridad')).astype(int) 
train['buenaUbicacion'] = (train.descripcion.str.contains('ubicacion') | train.descripcion.str.contains('ubicación')).astype(int)
train['lavadero'] = (train.descripcion.str.contains('lavadero') | train.descripcion.str.contains('lavado')).astype(int)
train['jacuzzi'] = (train.descripcion.str.contains('jacuzzi')).astype(int)


valor_adicionales = {'gimnasio':4,'usosmultiples':2,'piscina':5,'escuelascercanas':0.5,'centrosComercialescercanos':0.5,
                     'buenaUbicacion':2,'vigilancia': 5,'jardin':4,'lavadero':0.5}

def sumar_si(fila):
    for valor in fila:
        if valor == 1: return valor
    return 0
            
for columna in train.columns:
    if columna in valor_adicionales:
        train[columna] = train[columna].map(lambda x: x*valor_adicionales[columna])

train_2 = train.copy()
train_2['adicionales'] = train_2[bool_features].sum(axis=1)
promedio_adicionales = train_2.adicionales.mean()
desvio_standard = train_2.adicionales.std()
train_2['adicionales'] = (train_2.adicionales-promedio_adicionales)/desvio_standard
                                            
            

#train_2.head()

In [656]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [657]:
transformador_categorias = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
transformador_numeros = SimpleImputer()

In [658]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformador_numeros, num_features),
        ('cat', transformador_categorias, cat_features)
    ])

In [659]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=2000, learning_rate=0.05, max_depth=10, subsample_by_tree=0.5, colsample_by_col=0.5, min_child_weight=3,tree_method='gpu_hist', gpu_id=0)
from catboost import CatBoostRegressor
model2 = CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.05, loss_function='RMSE', task_type="GPU")

In [660]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [661]:
features = ['tipodepropiedad', 'antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'provincia','ciudad','adicionales'] 
train_y = train_2.precio.copy()
train_x = train_2[features].copy()

In [662]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y,train_size=0.2, test_size=0.8, random_state = 0)
my_pipeline.fit(train_x,train_y)
resultado_parcial = my_pipeline.predict(test_x)
from sklearn.metrics import mean_absolute_error
print("MAE XGBoost: ", mean_absolute_error(test_y, resultado_parcial))

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[16:54:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE XGBoost:  657471.719098664


In [663]:
pd.set_option('display.max_colwidth',-1)
import numpy as np

test.descripcion.fillna('',inplace=True)
test['jardin'] = (test.descripcion.str.contains('jardin') | test.descripcion.str.contains('jardín')).astype(int)
test['vigilancia'] = (test.descripcion.str.contains('vigilancia') | test.descripcion.str.contains('seguridad')).astype(int) 
test['buenaUbicacion'] = (test.descripcion.str.contains('ubicacion') | test.descripcion.str.contains('ubicación')).astype(int)
test['lavadero'] = (test.descripcion.str.contains('lavadero') | test.descripcion.str.contains('lavado')).astype(int)
test['jacuzzi'] = (test.descripcion.str.contains('jacuzzi')).astype(int)


valor_adicionales = {'gimnasio':4,'usosmultiples':2,'piscina':5,'escuelascercanas':0.5,'centrosComercialescercanos':0.5,
                     'buenaUbicacion':2,'vigilancia': 5,'jardin':4,'lavadero':0.5}

def sumar_si(fila):
    for valor in fila:
        if valor == 1: return valor
    return 0
            
for columna in test.columns:
    if columna in valor_adicionales:
        test[columna] = test[columna].map(lambda x: x*valor_adicionales[columna])

test_2 = test.copy()
test_2['adicionales'] = test_2[bool_features].sum(axis=1)
promedio_adicionales = test_2.adicionales.mean()
desvio_standard = test_2.adicionales.std()
test_2['adicionales'] = (test_2.adicionales-promedio_adicionales)/desvio_standard
                                            
            

#train_2.head()

NameError: name 'traitestn' is not defined

In [ ]:
my_pipeline.fit(train_x, train_y)

In [ ]:
resultado = my_pipeline.predict(test_2)
resultado

In [ ]:
res = pd.DataFrame(resultado, index=test.id, columns=['precio'])
res.reset_index(inplace=True)
res.columns = ["id", "target"]
display(res.head())
# RMSLE=1.0249284784393988 ?


# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "knn-results.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(res)